In [ ]:
#Install google earth engine api
!pip install earthengine-api
!pip install geemap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.4 MB/s eta 0:00:00


In [ ]:
import geemap
import pandas as pd
import numpy as np

In [ ]:
#Enable API here: https://console.cloud.google.com/apis/api/earthengine.googleapis.com
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project= 'wldfire-pred') #Please put your own project id


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [ ]:
#Define AOI (Area of Interest)
alberta = ee.FeatureCollection("FAO/GAUL/2015/level1") \
           .filter(ee.Filter.eq('ADM1_NAME', 'Alberta')) \
           .filter(ee.Filter.eq('ADM0_NAME', 'Canada'))

In [ ]:
#Define Date Range
start_date = '2010-01-01'
end_date = '2024-10-31'


In [ ]:
# Load weather data - ECMWF ERA5-Land Daily Aggregated Dataset (https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_DAILY_AGGR#bands)
climate_data = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR") \
                .filterDate(start_date, end_date) \
                .filterBounds(alberta)

# Air temperature and soil moisture
temperature_2m = climate_data.select('temperature_2m').map(lambda img: img.clip(alberta))
soil_moisture = climate_data.select('volumetric_soil_water_layer_1').map(lambda img: img.clip(alberta))

# Wind speed calculation
def calculate_wind_speed(image):
    u_wind = image.select('u_component_of_wind_10m') #East direction wind speed
    v_wind = image.select('v_component_of_wind_10m') #North direction wind speed
    wind_speed = u_wind.multiply(u_wind).add(v_wind.multiply(v_wind)).sqrt()
    return image.addBands(wind_speed.rename('wind_speed')).clip(alberta)

wind_speed_collection = climate_data.map(calculate_wind_speed)

# Define the relative humidity calculation function
def calculate_relative_humidity(image):
    temperature = image.select('temperature_2m')  # Temperature in Kelvin
    dewpoint = image.select('dewpoint_temperature_2m')  # Dewpoint in Kelvin

    # Calculate relative humidity using Earth Engine math operations
    numerator = dewpoint.multiply(17.625).divide(dewpoint.add(243.04)).exp()
    denominator = temperature.multiply(17.625).divide(temperature.add(243.04)).exp()
    rh = ee.Image(100).multiply(numerator.divide(denominator))
    return image.addBands(rh.rename('relative_humidity')).clip(alberta)

humidity_collection = climate_data.map(calculate_relative_humidity)


# Load Fire data
fire_data = ee.ImageCollection("MODIS/061/MOD14A1") \
                .filterDate(start_date, end_date) \
                .map(lambda img: img.clip(alberta))

fire_mask_vis = {

    'bands': ['MaxFRP', 'FireMask', 'FireMask']
}

# NDVI calculation
# Load the Landsat Tier 1 Collection for Surface Reflectance
landsat_collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
                      .filterDate('2024-03-01', '2024-10-31') \
                      .filterBounds(alberta)

# Function to mask clouds, cloud shadows, snow, and ice
def mask_clouds_snow(image):
    # Extract the QA_PIXEL band
    qa = image.select('QA_PIXEL')

    # Define mask conditions:
    # - Bit 3: Cloud (0 = no cloud, 1 = cloud)
    # - Bit 5: Snow (0 = no snow, 1 = snow)
    # - Bits 8-9: Cloud Confidence (00 = None, 01 = Low, 10 = Medium, 11 = High)
    # - Bits 12-13: Snow/Ice Confidence (00 = None, 01 = Low, 10 = Medium, 11 = High)

    cloud_mask = qa.bitwiseAnd(1 << 3).eq(0)  # No cloud
    snow_mask = qa.bitwiseAnd(1 << 5).eq(0)   # No snow
    cloud_confidence = qa.rightShift(8).bitwiseAnd(3).lte(1)  # Cloud confidence Low or below
    snow_confidence = qa.rightShift(12).bitwiseAnd(3).lte(1)  # Snow/Ice confidence Low or below

    # Combine mask conditions
    mask = cloud_mask.And(snow_mask).And(cloud_confidence).And(snow_confidence)

    # Apply the mask to remove cloud and snow pixels
    return image.updateMask(mask)

# Apply the mask to the collection and calculate NDVI
ndvi_collection = landsat_collection.map(mask_clouds_snow) \
                     .map(lambda img: img.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI').clip(alberta))

# Set NDVI visualization parameters
ndvi_vis_params = {
    'min': -1,
    'max': 1,
    'palette': ['brown', 'green']
}



In [ ]:
# Map visulization
Map = geemap.Map()
Map.centerObject(alberta, 6)
Map.addLayer(temperature_2m.first(), {'min': 250, 'max': 320, 'palette': ['blue', 'green', 'yellow', 'red']}, "2m Temperature")
Map.addLayer(soil_moisture.first(), {'min': 0, 'max': 1}, "Soil Moisture")
Map.addLayer(wind_speed_collection.select('wind_speed').first(), {'min': 0, 'max': 15, 'palette': ['blue', 'yellow', 'red']}, "Wind Speed")
Map.addLayer(fire_data, fire_mask_vis, 'Fire Mask')
Map.addLayer(humidity_collection.select('relative_humidity').first(), {'min': 0, 'max': 100, 'palette': ['blue', 'green', 'yellow', 'red']}, "Relative Humidity")
Map.addLayer(ndvi_collection.mean(), ndvi_vis_params, "NDVI (Landsat)")
Map.addLayer(alberta, {}, "Alberta")

# Show map
Map

Map(center=[54.985378674758095, -114.4053276398578], controls=(WidgetControl(options=['position', 'transparent…